In [58]:
import os
import pickle
import pandas as pd
import numpy as np
import networkx as nx


In [59]:
%%time

## Creating the mapping
## 1. transactionHash- Integer ID mapping

list_tx_hashes = []
path = r"C:\Users\Ashish Lakhmani\Downloads\edges2010"
for root, dir, files in os.walk(path):
    for f in files:
        with open(os.path.join(root,f), encoding = "utf-8") as f:
            for line in f:
                ## Split the row
                cells = line.split("\t")

                ## Add transaciton to Set
                transaction_hash = cells[1]
                list_tx_hashes.append(transaction_hash)
                
                
list_tx_hashes = list(set(list_tx_hashes))
txn_mapping = dict(zip(list_tx_hashes, range(len(list_tx_hashes))))
del list_tx_hashes

Wall time: 298 ms


In [60]:
%%time

## Creating the mapping
##  Address- Integer ID mapping

list_addresses = []
path = r"C:\Users\Ashish Lakhmani\Downloads\edges2010"
for root, dir, files in os.walk(path):
    for f in files:
        if("outputs" in f):
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    ## Split the row
                    cells = line.split("\t")

                    ## Add address to Set
                    addresses = cells[2:][::2]
                    list_addresses = list_addresses + addresses
                
                
list_addresses = list(set(list_addresses))
addr_mapping = dict(zip(list_addresses, range(len(list_addresses))))
del list_addresses

Wall time: 3min 11s


In [61]:
##Serialize Txn and Addr mappings

# import pickle
# path_txn_mapping2010 = r"C:\Users\japje\BDAProj\Working2010\txnHashtoID_mapping_2010.pickle"
# path_addr_mapping2010 = r"C:\Users\japje\BDAProj\Working2010\AddrToID_mapping_2010.pickle"
# with open(path_txn_mapping2010, "wb") as f:
#     pickle.dump(txn_mapping, f)

# with open(path_addr_mapping2010, "wb") as f1:
#     pickle.dump(addr_mapping, f1)
    

In [62]:
#testing parse logic

# sample_op_row = "1279128036	912c869c1ff6acaa9ca9d38ee66933c167ac980ee1c3e20086e3e6dd6a8896ef	18gBZnsuSrhYLjvPUgwvDUJmksfREUGBTT	1000000	1M5o4Es83Ls4UJ3md94FbLbebLhWzF8eQc	3079000000"

# cells = sample_op_row.split("\t")

In [63]:
%%time
## Generation of output mappings
## 1. txnID -> output_index -> OutputID
## 2. OutputID -> addressID -> amount

list_addresses = []
path = r"C:\Users\Ashish Lakhmani\Downloads\edges2010"
mapping_tx_to_op = {}
mapping_op = {}
global_index = 0
for root, dir, files in os.walk(path):
    for f in files:
        if("outputs" in f):
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    ## Split the row
                    cells = line.rstrip().split("\t")
                    # print(cells)
                    list_tx_index = list(range(0, len(cells[2:])//2))
                    list_addr_id = list(range(global_index+1, global_index+1+len(cells[2:])//2))
                    list_tx_rep = [txn_mapping[cells[1]]] * (len(cells[2:])//2)

                    list_addresses = [addr_mapping[x] for x in cells[2:][::2]]
                    list_amounts = cells[3:][::2]

                    mapping_delta1 = {x: {y: z} for x, y, z in zip(list_tx_rep, list_tx_index, list_addr_id)}

                    mapping_delta2 = {x: {y: z} for x, y, z in zip(list_addr_id, list_addresses, list_amounts)}

                    mapping_tx_to_op.update(mapping_delta1)
                    mapping_op.update(mapping_delta2)
                    global_index = global_index+1
                global_index = global_index+1




Wall time: 709 ms


In [64]:
len(mapping_op)

117489

In [65]:
 ## Serialize output mappings


path_txn_to_op_mapping2010 = r"C:\Users\Ashish Lakhmani\Downloads\edges2010\txnID_index_to_outputID_mapping_2010.pickle"
path_opID_to_addrAmt_mapping2010 = r"C:\Users\Ashish Lakhmani\Downloads\edges2010\opID_to_addrAmt_mapping2010.pickle"

with open(path_txn_to_op_mapping2010, "wb") as f:
    pickle.dump(mapping_tx_to_op, f)

with open(path_opID_to_addrAmt_mapping2010, "wb") as f1:
    pickle.dump(mapping_op, f1)

In [66]:
## Defining 2 node classes

# class Transaction:
#     def __init__(self, id):
#         self.id = id

# class Output:
#     def __init__(self, addr_id, amount):
#         self.addr_id = addr_id
#         self.amount = amount

In [72]:
## Reading inputs to create input mapping {transactionID --> [output indexes]}
## Adding edges

path = r"C:\Users\Ashish Lakhmani\Downloads\edges2010"
# mapping_tx_to_op = {}
mapping_ip = {}
for root, dir, files in os.walk(path):
    for f in files:
        if("inputs" in f):
            # print(f)
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    cells = line.rstrip().split("\t")
                    txn_id = txn_mapping[cells[1]]
                    
                    inputs = cells[2:]
                    input_index = []
                    for i in range(0, len(inputs)):
                        # print(str(inputs[i]) + "----"+str(inputs[i+1]))
                        try:
                            print(str(txn_mapping[inputs[i]]) + "----" + str(inputs[i+1]))
                            input_index.append(mapping_tx_to_op[int(txn_mapping[inputs[i]])][int(inputs[i+1])])
                        except:
                            pass
                        i = i+2
                    mapping_ip[txn_id] = input_index



91429----0
65237----0
43738----1
39362----0
62738----0
64085----0
100557----0
34176----0
72386----1
9624----0
115728----0
114806----0
100031----1
5257----1
115718----0
18175----0
16628----0
51889----0
20053----1
115728----1
67271----1
76323----0
114914----1
111108----0
100417----0
15075----0
62236----0
16788----0
107155----0
51576----0
20578----0
28214----0
18175----1
100417----1
114914----0
20053----0
100031----0
20578----1
32437----0
65237----1
72386----0
64085----1
41961----0
31153----0
8456----0
109398----0
6203----1
71537----1
107174----0
30694----1
1954----0
100155----0
25518----0
95129----0
89282----1
2954----0
102216----0
104904----0
72035----0
94662----1
116822----0
19833----1
62224----0
69695----1
36529----0
85569----0
33779----0
74251----0
3663----0
3760----0
26163----0
58836----0
11045----1
7017----0
103087----0
97873----0
19411----1
108723----0
115515----1
23467----0
4055----1
19138----0
25330----0
105289----0
35333----0
78641----0
75354----0
76743----0
117299----0
24215--

68271----1
91182----0
39189----0
18753----0
31749----0
6554----1
37156----0
234----0
2052----1
114937----0
77534----1
61994----0
87413----0
87132----1
68850----0
109788----1
67834----0
99733----1
72735----1
74055----0
1227----0
81540----1
77517----0
19442----0
48957----1
56648----0
98237----0
71974----1
11607----0
8616----0
114289----0
53316----1
69868----1
72144----1
52965----0
105603----0
51990----1
9771----0
77634----1
101758----0
116156----0
18227----0
38976----0
14803----0
89186----1
38821----1
104991----1
57729----1
57481----1
94510----0
65713----0
349----1
49858----1
113512----1
89061----1
86919----0
48695----0
7407----1
62890----1
115169----0
13572----0
44624----1
12053----0
59797----1
67590----0
116257----0
71315----0
32933----0
10535----0
21028----0
63247----0
52906----1
100014----0
52599----1
81686----0
38273----0
48068----1
107019----0
25697----1
12883----0
1312----0
65169----0
17012----1
40935----0
83188----1
84941----1
54613----0
22898----1
72851----0
94803----1
15543----

78004----1
48718----0
4978----1
78779----1
90231----1
29066----0
89028----1
60081----1
13822----0
74045----1
51935----1
76602----0
57882----0
76638----0
72072----0
8573----0
50955----0
68842----1
27337----1
92740----0
95673----1
29342----1
101489----1
17277----1
12407----0
115239----0
9440----0
57432----1
85253----0
24932----1
112438----0
17258----0
112742----1
70304----0
29330----1
37316----0
5633----0
117050----0
83816----0
45655----0
49425----0
57510----1
75719----1
98188----1
66049----0
7803----0
7803----1
70140----0
97445----1
27817----1
45153----0
28943----0
94110----1
113019----1
79511----1
41510----1
5900----0
81984----1
16828----1
62995----0
74052----0
533----1
47983----0
8188----1
18953----0
89263----1
107128----1
58147----0
91231----0
22504----1
18172----1
105207----1
18734----0
17266----1
76527----1
13188----0
19400----0
79662----0
58824----1
42217----1
43062----0
85448----1
111919----0
37732----1
105855----0
14065----0
12826----1
8387----1
43391----1
76269----0
5827----0
1

24319----0
24319----1
51452----1
41992----1
85789----1
87591----0
102668----0
88261----0
64981----0
64981----1
54071----1
97755----1
107609----0
82207----1
108314----0
31544----0
24492----1
70885----0
15482----0
87846----1
81739----0
82821----1
8925----1
97244----1
62460----1
34981----0
99442----1
51688----0
24786----1
47186----1
22057----0
35550----0
75747----1
6140----0
65790----0
67921----0
4116----1
63464----1
19833----0
81710----1
98384----1
40186----1
12194----0
100702----1
87379----0
66051----1
113568----1
3585----0
109294----1
109606----1
68797----0
56058----0
21995----0
110607----0
56278----0
49053----1
81854----1
40187----0
70127----1
74480----1
97750----0
26785----0
55104----1
12962----0
15023----0
58438----0
81548----0
75059----1
11017----1
16048----0
83082----0
28818----0
41710----0
74052----1
56176----1
111493----0
106899----0
51452----0
19616----0
67625----1
111446----1
23262----1
55193----1
116982----1
115545----1
45910----0
95927----0
1731----1
50541----1
91513----1
77

107941----0
94895----0
90584----1
104965----0
115033----0
83563----0
7109----0
31562----0
82666----1
57049----1
9580----1
55862----1
21670----1
80108----0
56250----1
33496----0
52747----0
51888----0
32671----0
47867----1
26698----1
112822----0
86852----0
98722----0
84702----0
59817----0
9341----0
100664----1
73067----0
27960----1
90342----1
51424----1
81136----1
85944----0
111392----0
86924----0
61967----0
61967----1
93746----0
63127----1
36620----0
18767----0
51272----0
110382----1
100321----1
25539----1
54680----1
36976----0
73862----1
64791----0
83395----0
7547----0
46625----0
31902----1
51252----0
31656----0
88497----1
86691----0
104453----0
48410----0
67750----0
67305----1
1103----1
56478----0
93064----1
115707----0
67509----0
29883----0
79521----0
32726----1
106861----0
37278----0
35921----1
26734----0
38265----0
36981----1
86937----1
58259----1
78345----1
29695----1
65626----1
115912----0
45110----0
41723----1
21403----1
25322----0
115315----0
90743----1
106411----0
374----1
827

...

60478----1
76952----0
54161----1
39040----1
21491----1
39371----0
93567----1
10535----1
114398----0
31376----1
6677----1
68108----1
60107----1
44532----1
88583----0
61378----1
8320----1
25480----1
44851----0
44851----1
41045----1
4445----1
31217----0
91622----0
54403----1
37268----0
106926----1
100292----0
66901----0
30971----0
114724----0
19826----1
109029----1
45579----1
94074----0
55532----0
96657----0
19952----0
28731----0
82002----1
33045----1
37830----1
100506----0
10235----0
115409----0
108368----1
1903----0
26546----0
20453----1
65329----0
106505----1
103213----0
63958----0
108619----0
36214----0
113886----0
89951----0
27417----0
111204----1
15112----1
87176----0
111291----1
10246----1
17315----1
60237----0
78929----0
91068----1
16131----1
6704----1
51978----0
65416----1
66595----0
5531----0
77573----0
104504----0
8566----1
56678----0
87420----1
73021----0
92647----0
92647----1
16164----0
26754----1
84418----0
73148----0
116139----1
105905----0
73491----1
67427----0
82526----0


72756----0
107481----1
13136----0
71984----0
71466----0
56875----0
86250----1
96457----0
96457----1
22199----1
40345----0
43876----0
100801----1
90224----1
58076----0
98684----1
71475----1
67355----0
24876----1
78746----0
34537----1
79048----0
13718----1
104136----0
113263----1
32609----1
113887----0
89071----1
14765----0
44259----0
63341----1
96993----0
13570----0
62741----0
14054----1
60979----0
78965----0
78965----1
29318----0
44455----0
62617----0
43326----0
63441----1
23126----0
80558----0
38084----0
16080----0
112521----0
17932----1
3387----0
19582----0
105342----0
52266----0
20737----1
77432----0
70830----0
89405----0
114723----0
92100----0
75148----0
81342----1
89012----1
65628----0
40817----1
87701----1
68653----1
23144----1
70661----0
146----1
108803----1
68471----0
90585----0
91442----0
20849----1
59732----1
90956----0
21983----0
56643----0
76731----0
80313----1
36021----1
55739----0
4558----0
55805----0
74322----1
12355----1
25378----1
13439----1
94973----0
98585----1
11145

60102----1
90521----1
70526----0
98289----0
35096----0
96659----0
41223----1
48057----0
9662----0
108550----1
38344----1
59820----0
65183----0
115598----0
43595----0
43595----1
13608----1
72509----0
85477----0
113512----0
14803----1
43389----0
24411----1
110086----0
29527----0
13830----0
9412----1
47101----1
10543----0
99045----1
15391----0
6412----0
2715----0
98818----0
24975----0
116856----0
29607----0
28951----1
52072----1
112997----0
3498----0
3498----1
56090----0
60154----0
37726----0
35425----0
97777----1
91918----0
13831----1
90030----1
64211----0
48762----0
90095----0
30155----1
17512----1
78390----0
11424----0
43057----1
16787----0
16106----0
66379----0
25660----0
100373----1
81960----1
89405----1
79291----1
91455----0
84504----0
32693----0
5949----1
12033----0
9696----0
19298----1
9029----1
101433----1
28093----1
79402----0
89389----0
6243----0
96087----0
96087----1
58440----1
63799----0
29366----0
110920----0
110920----1
94703----0
82665----0
113642----0
113642----1
104991--

99----0
63581----0
81617----0
14999----1
80999----1
9558----0
108189----0
83469----0
95952----0
69353----1
93521----0
38826----1
52062----1
76862----1
89473----1
85864----1
99118----1
39412----0
49458----0
50146----0
87830----0
87830----1
31589----0
73135----0
73135----1
49814----1
29227----1
12873----1
96128----1
89628----1
17732----0
110294----1
67860----0
112056----0
5818----0
24876----0
85139----0
86997----0
108909----0
110487----0
23389----0
214----0
8329----1
101267----0
22531----0
66807----0
52024----1
18104----0
26228----1
59665----0
71945----1
105352----0
95951----0
107484----0
33338----1
43887----0
51990----0
30179----1
9771----1
56461----0
110357----0
107067----1
50992----1
61117----1
110325----0
85857----0
109347----0
25102----0
76658----0
10378----1
26608----1
42250----0
115213----0
13987----0
16142----0
52761----0
34508----0
85483----1
28908----1
16539----0
2----0
100292----1
71482----1
95951----1
55360----1
59919----1
27037----1
33441----0
105620----0
96013----1
22422---

68383----1
9659----0
111981----0
8327----1
60977----0
71911----1
59208----0
65794----0
70874----1
80794----0
107520----1
111636----0
6831----0
107145----1
73212----1
104221----0
44338----0
5705----1
98122----0
1935----0
77020----1
84409----1
8926----0
47399----0
64842----1
101436----0
735----0
42868----0
90413----1
15510----1
32225----1
92319----1
88850----1
71795----1
92350----1
65665----0
72704----0
53728----0
106818----1
26481----0
50215----1
42930----1
84336----1
32065----1
97986----0
56188----0
8425----0
24422----0
78464----0
104484----1
65718----1
203----1
110075----1
30430----0
31314----0
90534----0
70891----1
41943----0
20240----1
24990----1
82690----0
80793----0
38403----1
88941----1
84490----0
108782----0
6933----0
50895----0
17676----0
28997----0
40751----0
61153----1
70906----1
87362----1
94260----0
23752----0
22226----0
62197----0
9272----1
68812----1
96995----1
76750----1
49675----1
96128----0
45126----1
74026----0
74026----1
22184----1
82755----0
63747----0
47806----0
96

In [73]:
 ## Serialize input mapping
import pickle
path_ip_mapping2010 = r"C:\Users\Ashish Lakhmani\Downloads\edges2010\inp_mapping_2010.pickle"


with open(path_ip_mapping2010, "wb") as f:
    pickle.dump(mapping_ip, f)



In [74]:
## 1. generate directed graph (NetworkX library) - Insert transactions (prefix "t-") and outputs from the output file

## Initialize
G = nx.DiGraph()

## Adding the transactions and outputs from output mappings
for k in mapping_tx_to_op:
    G.add_node("t_"+ str(k))
    connected_op = list(mapping_tx_to_op[k].values())
    # print(connected_op)
    G.add_nodes_from(connected_op)
    G.add_edges_from([("t_"+ str(k), y) for y in connected_op])
    
## Adding additional edges from input mapping
for k in mapping_ip:
    inputs = mapping_ip[k]
    # print([("t_"+ str(k), y) for y in inputs])
    G.add_edges_from([("t_"+ str(k), y) for y in inputs])


In [75]:
## Serialize Graph

nx.write_gpickle(G, r"C:\Users\Ashish Lakhmani\Downloads\edges2010\DiGraph.pickle")

In [83]:
# G[Transaction(12)].edges

# for n in G:
#     print(n.id)

#G[t1][o3]


In [84]:

# ##testing networkx

# ## 1. generate graph

# t1 = Transaction(12)
# t2 = Transaction(123)
# t3 = Transaction(43)

# o1 = Output(12, 3400000)
# o2 = Output(15, 34000000)
# o3 = Output(126, 34000004)

# G = nx.DiGraph()
# G.add_node(t1)
# G.add_node(t2)
# G.add_node(t3)

# G.add_node(o1)
# G.add_node(o2)
# G.add_node(o3)

# G.add_edge(t1, o2)
# G.add_edge(t2, o1)
# G.add_edge(t1, o3)
# G.add_edge(t3, o1) 


In [85]:
set([len(x) for x in mapping_ip.values()])

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 76,
 79,
 80,
 84,
 92,
 96,
 101,
 105,
 106,
 117,
 118,
 119,
 131,
 136,
 162,
 175,
 199,
 205,
 279,
 287,
 314,
 363,
 384,
 387,
 451,
 490}

In [86]:
mapping_tx_to_op[96434][1]

KeyError: 1